In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as pl
%matplotlib inline

In [ ]:
# set parameters

# cross-section in units-squared
sigma = 20
# number of particles 
N = 500
# size of box
l = 100
# maximum start velocity
maxv = 10
# timestep
dt = 0.1

In [ ]:
# set initial conditions 

# radius of a particle
r = np.sqrt(sigma/np.pi)
# random positions assuming square box of side-length l reaching from the origin to (x,y) = (l,l)
x = np.random.rand(N)*l
y = np.random.rand(N)*l
# random velocities
v = np.random.rand(N)*maxv
theta = np.random.rand(N)*2*np.pi
vx = v*np.cos(theta)
vy = v*np.sin(theta)

In [ ]:
# plot initial conditions

fig = pl.figure(figsize = (20, 4))
pl.subplot(131)
pl.hist(np.sqrt(vx*vx + vy*vy), bins=20);
pl.subplot(132)
pl.plot(x, y, '.')
pl.subplot(133)
pl.plot(vx, vy, '.')

In [ ]:
# function to enforce hard wall boundary conditions - does not conserve momentum in collisions with walls
def hard_wall(x, y, vx, vy):
    out = outside(x, y)
    if not out:
        return vx, vy
    if out == 1 or out == 2:
        return -vx, vy
    if out == 3 or out == 4:
        return vx, -vy

# function to enforce periodic boundaries
def periodic(x, y):
    out = outside(x, y)
    if not out:
        return x, y
    if out == 1 or out == 2:
        return x % l, y
    if out == 3 or out == 4:
        return x, y % l
    
# returns zero if the coordinates are inside the simulation space, and 1, 2, 3, or 4 if the particle is 
# outside the left, right, top, or bottom boundary respectively. 
def outside(x, y):
    if x < 0:
        return 1
    if x > l:
        return 2
    if y > l: 
        return 3
    if y < 0: 
        return 4

In [ ]:
# function to check for collisions
def coll_check(x1, y1, x2, y2):
    if dist(x1, y1, x2, y2) < r:
        return True
    else: 
        return False

# computes distance between two points
def dist(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2. + (y1 - y2)**2.)

# compute kinetic energy of a particle
def energy(vx, vy):
    return np.sum(0.5*(vx**2 + vy**2))

# move particles that are too close to each other initially - randomly. Better approach would be to define a potential 
# but that sounds like so much work... this might still leave some particles too close to each other. 
def relax(x, y):
    for i in range(len(x)):
        for j in range(i):
            if coll_check(x[i], y[i], x[j], y[j]):
                x[i] = np.random.rand()*l
                y[i] = np.random.rand()*l
                
# relax
for i in range(100):
    relax(x, y)

In [ ]:
# run the simulation

def step(x, y, vx, vy):
    for i in range(len(x)):
        # hard wall boundary conditions
        #vx[i], vy[i] = hard_wall(x[i], y[i], vx[i], vy[i])
        # periodic boundary conditions
        x[i], y[i] = periodic(x[i], y[i])
        for j in range(i):
            if coll_check(x[i], y[i], x[j], y[j]):
                
                # execute collision
                vi2 = vx[i]**2 + vy[i]**2
                vj2 = vx[j]**2 + vy[j]**2
                theta = np.arctan((y[i]-y[j])/(x[i]-x[j]))
                vi_norm = vx[i]*np.cos(theta) + vy[i]*np.sin(theta)
                vj_norm = vx[j]*np.cos(theta) + vy[j]*np.sin(theta)
                vi_tan = vx[i]*np.sin(theta) - vy[i]*np.cos(theta)
                vj_tan = vx[j]*np.sin(theta) - vy[j]*np.cos(theta)
                vi_norm_tmp = vi_norm
                vi_norm = vj_norm
                vj_norm = vi_norm_tmp
                vi2 = vi_norm**2 + vi_tan**2
                vj2 = vj_norm**2 + vj_tan**2
                vx[i] = vi_norm*np.cos(theta) + vi_tan*np.sin(theta)
                vy[i] = vi_norm*np.sin(theta) - vi_tan*np.cos(theta)
                vx[j] = vj_norm*np.cos(theta) + vj_tan*np.sin(theta)
                vy[j] = vj_norm*np.sin(theta) - vj_tan*np.cos(theta)
        # step particle positions        
        x[i] += vx[i]*dt
        y[i] += vy[i]*dt

In [ ]:
from matplotlib import animation

fig = pl.figure()
ax = pl.axes(xlim=(0, 100), ylim=(0, 100))
line, = ax.plot([], [], '.')

def init():
    line.set_data([],[])
    return line,

e = []
px = []
py = []
def animate(i):
    step(x, y, vx, vy)
    e.append(energy(vx, vy))
    px.append(np.sum(vx))
    py.append(np.sum(vy))
    line.set_data(x, y)
    return line,

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)
anim.save('particles.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

In [ ]:
fig = pl.figure(figsize=(20, 4))
pl.subplot(131)
pl.hist(np.sqrt(vx*vx + vy*vy), bins=20);
pl.xlabel('velocity')
pl.ylabel('N(v)')
pl.title('velocity distribution of particles')
pl.subplot(132)
pl.plot(py)
pl.plot(px)
pl.xlabel('timesteps')
pl.ylabel('momentum')
pl.title('momentum conservation')
pl.subplot(133)
pl.plot(e)
pl.xlabel('timesteps')
pl.ylabel('energy')
pl.title('energy conservation\n')